In [1]:
from IPython.display import clear_output
!pip3 install -U lazypredict
!pip3 install -U pandas #Upgrading pandas
!pip3 install -U scikit-learn #Upgrading pandas

clear_output()

In [2]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
#Import packages
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier

import time

In [4]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [5]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S


In [6]:
print(f'\033[94mNumber of rows in train data: {df_train.shape[0]}')
print(f'\033[94mNumber of columns in train data: {df_train.shape[1]}')
print(f'\033[94mNumber of values in train data: {df_train.count().sum()}')
print(f'\033[94mNumber missing values in train data: {sum(df_train.isna().sum())}')

Number of rows in train data: 891
Number of columns in train data: 12
Number of values in train data: 9826
Number missing values in train data: 866


In [7]:
#Checking null rates of columns
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
#Basic numeric stats about dataset
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.00,891.00,891.00,714.00,891.00,891.00,891.00
mean,446.00,0.38,2.31,29.70,0.52,0.38,32.20
std,257.35,0.49,0.84,14.53,1.10,0.81,49.69
min,1.00,0.00,1.00,0.42,0.00,0.00,0.00
25%,223.50,0.00,2.00,20.12,0.00,0.00,7.91
50%,446.00,0.00,3.00,28.00,0.00,0.00,14.45
75%,668.50,1.00,3.00,38.00,1.00,0.00,31.00
max,891.00,1.00,3.00,80.00,8.00,6.00,512.33


## EDA

In [9]:
df_train.drop(["PassengerId"],axis=1,inplace=True)
#df_test.drop(["PassengerId"],axis=1,inplace=True)
y_col = "Survived"

In [10]:
df_train.iloc[:,1:]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.25,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.28,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.92,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.10,C123,S
4,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.05,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.00,0,0,211536,13.00,NaN,S
887,1,"Graham, Miss. Margaret Edith",female,19.00,0,0,112053,30.00,B42,S
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,1,"Behr, Mr. Karl Howell",male,26.00,0,0,111369,30.00,C148,C


In [11]:
df_train.iloc[:, 1:].describe().T.sort_values(by='std' , ascending = False)\
                     .style.background_gradient(cmap='GnBu')\
                     .bar(subset=["max"], color='#BB0000')\
                     .bar(subset=["mean",], color='green')

,count,mean,std,min,25%,50%,75%,max
Fare,891.00,32.20,49.69,0.00,7.91,14.45,31.00,512.33
Age,714.00,29.70,14.53,0.42,20.12,28.00,38.00,80.00
SibSp,891.00,0.52,1.10,0.00,0.00,0.00,1.00,8.00
Pclass,891.00,2.31,0.84,1.00,2.00,3.00,3.00,3.00
Parch,891.00,0.38,0.81,0.00,0.00,0.00,0.00,6.00


## Null rates

In [12]:
train_null = pd.DataFrame(df_train.isna().sum()/df_train.shape[0])
test_null = pd.DataFrame(df_test.isna().sum()/df_test.shape[0])
test_null = test_null.sort_values(by = 0 ,ascending = False)
train_null = train_null.sort_values(by = 0 ,ascending = False)

In [13]:
#Plot null rates

fig = make_subplots(rows=1, 
                    cols=2,
                    column_titles = ["Train Data", "Test Data"] ,
                    x_title="Missing Values")

fig.add_trace(go.Bar(x=train_null[0],
                     y=train_null.index,
                     orientation="h",
                    marker=dict(color=[n for n in range(12)], 
                                line_color='rgb(0,0,0)' , 
                                line_width = 2,
                                coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(x=test_null[0], 
                     y=test_null.index,
                     orientation="h",
                    marker=dict(color=[n for n in range(12)], 
                                line_color='rgb(0,0,0)', 
                                line_width = 2,
                                coloraxis="coloraxis")),
              1, 2)

fig.update_layout(showlegend=False, title_text="Column wise Null Value Distribution", title_x=0.5)

In [14]:
train_null_rows = pd.DataFrame(df_train.isna().sum(axis=1).value_counts()/df_train.shape[0]).reset_index()
train_null_rows.columns = ['num_nulls', 'proportion']
test_null_rows = pd.DataFrame(df_test.isna().sum(axis=1).value_counts()/df_test.shape[0]).reset_index()
test_null_rows.columns = ['num_nulls', 'proportion']

#Make plots
fig = make_subplots(rows=1, 
                    cols=2,
                    column_titles = ["Train Data", "Test Data"] ,
                    x_title="Missing Values",)

fig.add_trace(go.Bar(x=train_null_rows["num_nulls"], 
                     y=train_null_rows["proportion"]  ,
                    marker=dict(color=[n for n in range(4)], 
                                line_color='rgb(0,0,0)' ,
                                line_width = 3
                                ,coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(x= test_null_rows["num_nulls"], 
                     y=test_null_rows["proportion"],
                    marker=dict(color=[n for n in range(4)], 
                                line_color='rgb(0,0,0)',
                                line_width = 3 ,
                                coloraxis="coloraxis")),
              1, 2)
fig.update_layout(showlegend=False, title_text="Row wise Null Value Distribution", title_x=0.5)

In [15]:
##Look at distribution of ages
train_age = df_train.copy()
test_age = df_test.copy()
train_age["type"] = "Train"
test_age["type"] = "Test"
ageDf = pd.concat([train_age, test_age])
fig = px.histogram(data_frame = ageDf, 
                   x="Age",
                   color= "type",
                   color_discrete_sequence =  ['#58D68D','#DE3163'],
                   marginal="box",
                   nbins= 100,
                    template="plotly_white"
                )
fig.update_layout(title = "Distribution of Age" , title_x = 0.5)
fig.show()

In [16]:
# Look at target distribution
target_df = pd.DataFrame(df_train['Survived'].value_counts()).reset_index()
target_df.columns = ['Survived', 'count']
fig = px.bar(data_frame =target_df, 
             x = 'Survived',
             y = 'count'
            ) 
fig.update_traces(marker_color =['#58D68D','#DE3163'], 
                  marker_line_color='rgb(0,0,0)',
                  marker_line_width=2,)
fig.update_layout(title = "Target Distribution",
                  template = "plotly_white",
                  title_x = 0.5)
print("\033[94mPercentage of Transported = 0: {:.2f} %".format(target_df["count"][0] *100 / df_train.shape[0]))
print("\033[94mPercentage of Transported = 1: {:.2f} %".format(target_df["count"][1]* 100 / df_train.shape[0]))
fig.show()

Percentage of Transported = 0: 61.62 %
Percentage of Transported = 1: 38.38 %


In [17]:
fig = px.imshow(df_train.corr() ,text_auto=True, aspect="auto" , color_continuous_scale = "viridis")
fig.show()

In [18]:
#Imputing age values
imputer_cols = ["Age"]

imputer = SimpleImputer(strategy='mean')
imputer.fit(df_train[["Age"]])
df_train[["Age"]] = imputer.transform(df_train[["Age"]])
df_test[["Age"]] = imputer.transform(df_test[["Age"]])

#Impute fare value
imputer.fit(df_train[["Fare"]])
df_test[["Fare"]] = imputer.transform(df_test[["Fare"]])

#Impute embarked value
imputer = SimpleImputer(strategy="most_frequent")
imputer.fit(df_train[["Embarked"]])
df_train[["Embarked"]] = imputer.transform(df_train[["Embarked"]])

In [19]:
combined = [df_train,df_test]

In [20]:
for data in combined:
    data["Title"] = data["Name"].str.extract(r'\s([A-Za-z]+)\.',expand=False)
    
    data['Title'] = data['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
                                              'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Ms', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')

In [21]:
for data in combined:
    data['is_female'] = data['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
    

In [22]:
enc = OneHotEncoder(handle_unknown = 'ignore',sparse=False)

In [23]:
enc.fit(df_train['Embarked'].to_numpy().reshape(-1,1))
ohe_array = enc.transform(df_train['Embarked'].to_numpy().reshape(-1, 1))
#Create a Pandas DataFrame of the hot encoded column
ohe_df = pd.DataFrame(ohe_array, columns=enc.get_feature_names_out(['Embarked']))
#concat with original data
df_train = pd.concat([df_train, ohe_df], axis=1)

ohe_array = enc.transform(df_test['Embarked'].to_numpy().reshape(-1, 1))
#Create a Pandas DataFrame of the hot encoded column
ohe_df = pd.DataFrame(ohe_array, columns=enc.get_feature_names_out())
#concat with original data
df_test = pd.concat([df_test, ohe_df], axis=1)

In [24]:
enc.fit(df_train['Title'].to_numpy().reshape(-1,1))
ohe_array = enc.transform(df_train['Title'].to_numpy().reshape(-1, 1))
#Create a Pandas DataFrame of the hot encoded column
ohe_df = pd.DataFrame(ohe_array, columns=enc.get_feature_names_out(['Title']))
#concat with original data
df_train = pd.concat([df_train, ohe_df], axis=1)

ohe_array = enc.transform(df_test['Title'].to_numpy().reshape(-1, 1))
#Create a Pandas DataFrame of the hot encoded column
ohe_df = pd.DataFrame(ohe_array, columns=enc.get_feature_names_out())
#concat with original data
df_test = pd.concat([df_test, ohe_df], axis=1)

In [25]:
#Drop cols no longer needed
combined = [df_train,df_test]
for data in combined:
    data.drop(columns=['Name','Sex','Ticket','Cabin','Embarked','Title'],inplace=True)

In [26]:
X = df_train.drop('Survived' , axis =1 )
y = df_train['Survived']
X_train , X_test , y_train , y_test = train_test_split(X , 
                                                       y, 
                                                       random_state = 12 ,
                                                       test_size =0.33)

In [27]:
clf = LazyClassifier(verbose=0,
                     ignore_warnings=True,
                     custom_metric=None,
                     predictions=False,
                     random_state=12,
                     classifiers='all')

models, predictions = clf.fit(X_train , X_test , y_train , y_test)

100%|██████████| 29/29 [00:01<00:00, 14.75it/s]


In [28]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LogisticRegression,0.81,0.79,0.79,0.81,0.02
LinearSVC,0.80,0.79,0.79,0.80,0.06
CalibratedClassifierCV,0.80,0.79,0.79,0.80,0.17
NuSVC,0.80,0.79,0.79,0.80,0.03
SVC,0.81,0.79,0.79,0.81,0.03
LinearDiscriminantAnalysis,0.80,0.78,0.78,0.80,0.03
RidgeClassifierCV,0.80,0.78,0.78,0.80,0.01
RidgeClassifier,0.80,0.78,0.78,0.80,0.02
PassiveAggressiveClassifier,0.79,0.78,0.78,0.79,0.01


## Logistic regression

In [29]:
scores = []
#feat_imp_list = []

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
for fold_no, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
    
    start_time = time.time()
    
    X_train = X.iloc[train_idx]
    X_valid = X.iloc[valid_idx]
    y_train = y.iloc[train_idx]
    y_valid = y.iloc[valid_idx]
    
    #Fit logistic regression model
    model = LogisticRegression()
    model.fit(X_train,y_train)
    
    #Predict for validation set
    valid_pred = model.predict(X_valid)
    
    #Calculate metrics
    acc = accuracy_score(y_valid,  valid_pred)
    
    scores.append(acc)
    run_time = time.time() - start_time
    
#     fi = pd.DataFrame(index=X.columns,
#                  data=model.feature_importances_,
#                  columns=[f'{fold_no}_importance'])
#     feat_imp_list.append(fim)
    
    print(f"Fold={fold_no+1}, Accuracy score: {acc:.2f}%, Run Time: {run_time:.2f}s")
    

Fold=1, Accuracy score: 0.82%, Run Time: 0.03s
Fold=2, Accuracy score: 0.84%, Run Time: 0.03s
Fold=3, Accuracy score: 0.81%, Run Time: 0.03s
Fold=4, Accuracy score: 0.80%, Run Time: 0.03s
Fold=5, Accuracy score: 0.87%, Run Time: 0.03s
Fold=6, Accuracy score: 0.85%, Run Time: 0.03s
Fold=7, Accuracy score: 0.81%, Run Time: 0.03s
Fold=8, Accuracy score: 0.83%, Run Time: 0.03s
Fold=9, Accuracy score: 0.82%, Run Time: 0.03s
Fold=10, Accuracy score: 0.82%, Run Time: 0.03s
Fold=11, Accuracy score: 0.82%, Run Time: 0.03s
Fold=12, Accuracy score: 0.83%, Run Time: 0.03s
Fold=13, Accuracy score: 0.85%, Run Time: 0.03s
Fold=14, Accuracy score: 0.84%, Run Time: 0.03s
Fold=15, Accuracy score: 0.80%, Run Time: 0.04s


In [30]:
#Grid search for hyperparameters
model=LogisticRegression()
param_grid = {
    'solver':['newton-cg', 'lbfgs', 'liblinear'],
    'penalty':['l2'],
    'C':[100, 10, 1.0, 0.1, 0.01]
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.827914 using {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.826041 (0.022321) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.827540 (0.022731) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.826041 (0.022321) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.827163 (0.022351) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.827914 (0.024000) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.826414 (0.021617) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.825292 (0.020040) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.827159 (0.019257) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.826033 (0.019492) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.818559 (0.021825) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.818185 (0.020644) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.812194 (0.018324) with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.783012 (0.014347) with: {'

## Model prediction for submission

In [31]:
model = LogisticRegression(C= 10, penalty = 'l2', solver= 'lbfgs')
model.fit(X,y)
submission_predictions = pd.DataFrame(model.predict(df_test.drop(columns = ['PassengerId'])))

In [32]:
submission_predictions.columns = ['Survived']
submission_predictions['PassengerId'] = df_test['PassengerId']

In [33]:
submission_predictions.to_csv("submission.csv",index=False)